In [2]:
import pandas as pd
import numpy as np
import sys
import os
from copy import deepcopy
import xlrd

In [ ]:
def poe(S_Ac,S_Dc, A_D):
    for sga in A_D.index:
        print(sga)
        d_l = [deg for deg in A_D.columns if A_D.loc[sga, deg]==1]
        all_fq = np.sum(S_Dc[d_l]).values
        t_fq = np.dot(S_Ac[sga].T, S_Dc[d_l])
        f_fq = np.dot(1-S_Ac[sga].T, S_Dc[d_l])
        print(np.mean(all_fq/(S_Ac.shape[0])))
        print(np.mean(t_fq/np.sum(S_Ac[sga])))
        print(np.mean(f_fq/np.sum(1-S_Ac[sga])))

def prei(S_Ado, S_Dm, A_D, S_Ac, S_Dc, A_Dc, e=1e-5, cut_value=0.85):

    S_Ad = S_Ado.values
    A_Dd = A_D.values
    S_Dd = S_Dm.values
    
    S_At = S_Ac.values
    S_Dt = S_Dc.values
    A_Dt = A_Dc.values
    
    idx = [list(A_D.index).index(ele) for ele in list(A_Dc.index)]
    col = [list(A_D.columns).index(ele) for ele in list(A_Dc.columns)]
    

    pA1 = ((np.sum(S_Ac, 0)[:, None] + 1) / (S_Ac.shape[0] + 2))
    pD1A1 = ((np.dot(S_Ad.T, S_Dd) + 1) / (np.sum(S_Ad, 0)[:, None] + 2))[idx, :][:,col]
    pD1A0 = ((np.dot(1 - S_Ad.T, S_Dd) + 1) / (np.sum(1 - S_Ad, 0)[:, None] + 2))[idx,:][:,col]
    
    logpA1 = np.log(pA1 + e)
    logpA0 = np.log(1 - pA1 + e)
    logpD1A1 = np.log(pD1A1 + e) * A_Dt
    logpD0A1 = np.log(1 - pD1A1 + e) * A_Dt
    logpD1A0 = np.log(pD1A0 + e) * A_Dt
    logpD0A0 = np.log(1 - pD1A0 + e) * A_Dt

    logpA1D = np.dot(S_Dt, logpD1A1.T) + np.dot(1 - S_Dt, logpD0A1.T) + logpA1.T
    logpA0D = np.dot(S_Dt, logpD1A0.T) + np.dot(1 - S_Dt, logpD0A0.T) + logpA0.T
    S_Ad = 1 / (1 + np.exp(logpA0D - logpA1D))

    S_Ad0 = pd.DataFrame(S_Ad, index=S_Ac.index, columns=S_Ac.columns)
    S_Ad1 = deepcopy(S_Ad0)
    S_Ad1[S_Ad1 > cut_value] = 1
    S_Ad1[S_Ad1 <= cut_value] = 0
    
    prm = [logpA1, logpA0, logpD1A1, logpD0A1, logpD1A0, logpD0A0]
    
    return S_Ad0, S_Ad1, prm

In [3]:
S_Am = pd.read_csv('Pre/S_Amrn.csv', header=0, index_col=0)
S_Dm = pd.read_csv('Pre/S_Dmrn.csv', header=0, index_col=0)
A_Dm = pd.read_csv('Pre/A_Drn.csv', header=0, index_col=0)

In [ ]:
def sta(input_name, output_name, tumor_type, cut):
    
    # input_name
    if 'All' in input_name:
        mut = pd.read_csv('EMva/Input/S_Ac.csv', header=0, index_col=0)
        pred = pd.DataFrame(np.ones((mut.shape[0], mut.shape[1])), index=mut.index, columns=mut.columns)
        
    if 'Pathway' in input_name:
        mut = pd.read_csv('EMva/Input/S_Ac.csv', header=0, index_col=0)
        pred = pd.read_csv(input_name, header=0, index_col=None)
        pred.columns = [ele[4:] for ele in pred.columns]
        pred.index = mut.index
        new_col = mut.columns & pred.columns
        pred = pred[new_col]
        
    if 'S_A' in input_name:
        pred = pd.read_csv(input_name, header=0, index_col=0)
        
    label = pd.read_csv('DrugNormal/Input/GDSC drug response.csv', header=0, index_col=0)
    label = label.loc[pred.index, :]
    
    # tumor_type
    if tumor_type == 1:
        ttcn = pd.read_csv('DrugNormal/Input/ttcn.csv', header=0, index_col=0)
        tt_row = [ele for ele in pred.index if ele in ttcn.iloc[:, 0].values]
        pred = pred.loc[tt_row]
        label = label.loc[tt_row]
    
    
    drug_l = ['171','326','228','1053','86',
              '1502','1030','152',
              '1061','1036','1371','1373','29','159','30',
              '292','293','5','298',
              '54','1054','219',
              '1','1010','1114','282','119','1032','1377','273','1143',
              '255','1049','155','308',
              '269','269-1047','1133-1047',
              '134',
              '1067',
              '223-1066-224-238','1058-1066-224-238','1527-1066-224-238','302-1066-224-238','283-1066-224-238','1057-1066-224-238',
              '3','83','299','1016','1059','1166']
    
    gene_l = ['AKT1','AKT1','AKT1','AKT1','AKT1',
              'AR','ATM','ATM',
              'BRAF','BRAF+RAF1','BRAF+RAF1','BRAF+RAF1','BRAF+RAF1','BRAF','BRAF+RAF1+PDGFRA+KIT',
              'KIT+PDGFRA','KIT+PDGFRA','KIT+PDGFRA','KIT+KDR',
              'CDK4','CDK4','CDK4',
              'EGFR','EGFR','EGFR','EGFR','EGFR+ERBB2','EGFR+ERBB2','EGFR+ERBB2','EGFR+ERBB2','EGFR',
              'ERBB2','FGFR1+FGFR3','FGFR1+FGFR3+PDGFRA','FGFR1+FGFR3+PDGFRA+KDR',
              'MDM4','MDM4','TP53',
              'TOP2A',
              'PPM1D',
              'PIK3CA','PIK3CA','PIK3CA','PIK3CA','PIK3CA+MTOR','PIK3CA+MTOR',
              'MTOR','MTOR','MTOR','MTOR','MTOR','MTOR']
    
    bdf = np.zeros((label.shape[0],0))
    for drug in drug_l:
        if '-' in drug:
            drug_sl = drug.split('-')
            for i in range(len(drug_sl)):
                if i == 0:
                    df = label[[drug_sl[i]]].values
                else:
                    df = df - label[[drug_sl[i]]].values
            bdf = np.concatenate((bdf, df),axis=1)
        else:
            df = label[[drug]].values
            bdf = np.concatenate((bdf, df),axis=1)
    bdf[bdf<0] = 0
    label_col = [gene_l[i] + '-' +str(i) for i in range(len(gene_l))]
    label = pd.DataFrame(bdf, index=label.index, columns=label_col)

    bdf = np.zeros((label.shape[0],0))
    for gene in gene_l:
        if '+' in gene:
            gene_sl = gene.split('+')
            for i in range(len(gene_sl)):
                if i == 0:
                    df = pred[[gene_sl[i]]].values
                else:
                    df = df + pred[[gene_sl[i]]].values
            bdf = np.concatenate((bdf, df),axis=1)
        else:
            df = pred[[gene]].values
            bdf = np.concatenate((bdf, df),axis=1)
    bdf[bdf>1] = 1
    pred_col = [gene_l[i] + '-' +str(i) for i in range(len(gene_l))]
    pred = pd.DataFrame(bdf, index=label.index, columns=pred_col)
    
    pred = pred[label>=0]
    
    label.to_csv('Heatmap/label.csv', index=True, header=True)
    pred.to_csv('Heatmap/pred.csv', index=True, header=True)
    # normal f1
    label_pred = label+pred
    label_pred[label_pred>1]=1
    label_sum = np.sum(label, 0)
    pred_sum = np.sum(pred, 0)
    tp = np.sum((label+ pred) - label_pred,0)
    precision = tp / pred_sum
    recall = tp / label_sum
    F1 = 2 * precision * recall / (precision + recall)
    
    out_put = pd.concat((tp, precision, recall, F1), axis=1)
    out_put.columns = ['TP', 'Precision', 'Recall', 'F1']
    out_put.to_csv(output_name, header=True, index=True)
    return out_put

def f1():
    # micro f1
    from sklearn.metrics import f1_score
    f1 = []
    for i in range(pred.shape[1]):
        val_l = [ele for ele in pred.iloc[:,i].tolist() if np.isnan(ele) == False ]
        preds = pred.iloc[val_l,i].tolist()
        labels = label.iloc[val_l,i].tolist()
        f1.append(f1_score(labels, preds, average='macro'))
    f1_df = pd.DataFrame(f1, index=pred.columns, columns=['f1'])

In [ ]:
def dr_sen(S_Ac, S_Dc, A_Dc, func, fix_mutation=0, e=1e-5, cut_value=0.5, ite=10000, cover_thres=1e-4):

    # em
    if func == 'mut':
        mut_op = sta('EMva/Input/S_Ac.csv', 'EMva/Output/sta/S_Ac.csv', 0, 0.5)

    # use TCGA DEG run TCGA EM, done
    if func == 'ttg':
        S_Ad = pd.read_csv('EMva/Output/S_Ad1.csv', header=0, index_col=0)
        S_Dm = pd.read_csv('EMva/Input/S_Dm.csv', header=0, index_col=0)
        S_Attg0, S_Attg1, prm = prei(S_Ad, S_Dm, A_D, S_Ac, S_Dc, A_Dc)
        S_Attg1.to_csv('EMva/Output/S_Attg1.csv', header=True, index=True)
        sta('EMva/Output/S_Attg1.csv', 'EMva/Output/sta/S_Attg1.csv', 0, 0.5)

    # use GDSC DEG run TCGA EM, done
    if func == 'tgg':
        S_Am = pd.read_csv('EMva/Input/S_Am.csv', header=0, index_col=0)
        S_Dm = pd.read_csv('EMva/Input/S_Dm.csv', header=0, index_col=0)
        S_Atgg0, S_Atgg1, prm = EM(S_Am.loc[:, A_Dc.index], S_Dm.loc[:,
                                                            A_Dc.columns], A_D.loc[A_Dc.index, :], fix_mutation, e,
                                   cut_value, ite, cover_thres)
        S_Atgg1 = prei(S_Atgg1, S_Dm, A_D, S_Ac, S_Dc, A_Dc)
        S_Atgg1.to_csv('EMva/Output/S_Atgg1.csv', header=True, index=True)

    # use GDSC run EM
    if func == 'ggg':
        S_Aggg0, S_Aggg1, prm = EM(S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
        S_Aggg1.to_csv('EMva/Output/S_Aggg1.csv', header=True, index=True)
        S_Aggg1_sta = sta('EMva/Output/S_Aggg1.csv', 'EMva/Output/sta/S_Aggg1.csv', 0, 0.5)

    # use GDSC run 10 EM
    if func == 'ggg10':
        S_Agggt0, S_Agggt1 = EM10fold(S_Ac, S_Dc, A_Dc)
        S_Agggt1.to_csv('EMva/Output/S_Aggg1.csv', header=True, index=True)
        S_Aggg1_sta = sta('EMva/Output/S_Aggg1.csv', 'EMva/Output/sta/S_Aggg1.csv', 0, 0.5)

    # use GDSC run fges
    if func == 'ggg_fges':
        S_Aggg0, S_Aggg1, prm = EM(S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
        
        A_Dc.index = ['sga:' + idx for idx in A_Dc.index]
        A_Dc.columns = ['dge:' + col for col in A_Dc.columns]
        S_Ac.columns = A_Dc.index
        S_Dc.columns = A_Dc.columns
        S_Aggg1.columns = A_Dc.index
        S_Aggg1.to_csv('EMva/Input/S_Aggg1.csv', header=True, index=True)
        S_Ac.to_csv('EMva/Input/S_Ac_fges.csv', header=True, index=True)
        S_Dc.to_csv('EMva/Input/S_Dc_fges.csv', header=True, index=True)
        A_Dc.to_csv('EMva/Input/A_Dc_fges.csv', header=True, index=True)
        
        os.system('python fges_stem.py EMva/Input/S_Ac_fges.csv EMva/Input/S_Dc_fges.csv EMva/Input/A_Dc_fges.csv noEM EMva/Output/S_Aggg1.csv PI3K Pathway/GDSC 30')

    # random A_D
    if func == 'ggg_rad':
        stat = pd.DataFrame(np.zeros((52,4)))
        for i in range(10):
            A_D = pd.read_csv('EMva/Input/A_D_rd/A_D_rd'+'_'+str(i)+'.csv', header=0, index_col=0)
            A_Dc = A_D.loc[S_Ac.columns, A_D.columns & S_Dc.columns]
            print(np.sum(A_Dc, 1))
            S_Ad0_rad, S_Ad1_rad, prm = EM(S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
            S_Ad1_rad.to_csv('EMva/Output/A_D_rd/S_Ad1_rad'+'_'+str(i)+'.csv', header=True, index=True)

            out_put = sta('EMva/Output/A_D_rd/S_Ad1_rad'+'_'+str(i)+'.csv', 'EMva/Output/sta/A_D_rd/S_Ad1_rad'+'_'+str(i)+'.csv', 0, 0.5)
            stat.index=out_put.index
            stat.columns = out_put.columns
            stat = stat+out_put
        
        stat = stat/10
        stat.to_csv('EMva/Output/sta/S_Ad1_rad_mean.csv', header=True, index=True)

    if func == 'ggg_rad10':
        stat = pd.DataFrame(np.zeros((52,4)))
        for i in range(10):
            print(i)
            A_D = pd.read_csv('EMva/Input/A_D_rd/A_D_rd'+'_'+str(i)+'.csv', header=0, index_col=0)
            A_Dc = A_D.loc[S_Ac.columns, A_D.columns & S_Dc.columns]
            
            S_Ad0_rad, S_Ad1_rad = EM10fold(S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
            S_Ad1_rad.to_csv('EMva/Output/A_D_rd/S_Ad1_rad'+'_'+str(i)+'.csv', header=True, index=True)
            
            out_put  = sta('EMva/Output/A_D_rd/S_Ad1_rad'+'_'+str(i)+'.csv', 'EMva/Output/sta/A_D_rd/S_Ad1_rad'+'_'+str(i)+'.csv', 0, 0.5)
            stat.index=out_put.index
            stat.columns = out_put.columns
            stat = stat+out_put
        
        stat = stat/10
        stat.to_csv('EMva/Output/sta/S_Ad1_rad_mean.csv', header=True, index=True)

    # random A_D fges
    if func == 'ggg_rad_fges':
        A_D = pd.read_csv('EMva/Input/A_D_rd.csv', header=0, index_col=0)
        A_Dc = A_D.loc[S_Ac.columns, A_D.columns & S_Dc.columns]
        
        S_Ad0_rad, S_Ad1_rad, prm = EM(S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
        S_Ad1_rad.to_csv('EMva/Output/S_Ad1_rad.csv', header=True, index=True)
        
        A_Dc.index = ['sga:' + idx for idx in A_Dc.index]
        A_Dc.columns = ['dge:' + col for col in A_Dc.columns]
        
        S_Ad1_rad.columns = A_Dc.index
        S_Ad1_rad.to_csv('EMva/Input/S_Ad1_rad.csv', header=True, index=True)
        A_Dc.to_csv('EMva/Input/A_Dc_rd_fges.csv', header=True, index=True)
        
        os.system('python fges_mcmc.py EMva/Input/S_Ac_fges.csv EMva/Input/S_Dc_fges.csv EMva/Input/A_Dc_rd_fges.csv noEM EMva/Output/S_Ad1_rad.csv PI3K Pathway/GDSC_rad 30')
        
    # random S_A
    if func == 'ggg_rsa':
        S_Ac = pd.read_csv('EMva/Input/S_Ac_rd.csv',header=0, index_col=0).T
        print(np.sum(S_Ac, 0))
        
        S_Ad0_ra, S_Ad1_ra, prm = EM(S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
        S_Ad1_ra.to_csv('EMva/Output/S_Ad1_rsa.csv', header=True, index=True)
    
    # random S_D
    if func == 'ggg_rsd':
        stat = pd.DataFrame(np.zeros((52,4)))
        for i in range(10):
            S_Dc = pd.read_csv('EMva/Input/S_Dch_rd/S_Dc_rd'+'_'+str(i)+'.csv', header=0, index_col=0)
            S_Dc = S_Dc.loc[S_Ac.index, A_Dc.columns]
            S_Dc = S_Dc.reset_index().drop_duplicates(subset='index',keep='first').set_index('index')
            S_Ad0_rsd, S_Ad1_rsd, prm = EM(S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
            S_Ad1_rsd.to_csv('EMva/Output/S_Dc_rd/S_Ad1_rsd'+'_'+str(i)+'.csv', header=True, index=True)        

            out_put = sta('EMva/Output/S_Dc_rd/S_Ad1_rsd'+'_'+str(i)+'.csv', 'EMva/Output/sta/S_Dc_rd/S_Ad1_rsd'+'_'+str(i)+'.csv', 0, 0.5)
            pinr(np.sum(S_Ad1_rsd))
            stat.index=out_put.index
            stat.columns = out_put.columns
            stat = stat+out_put
        
        stat = stat/10
        stat.to_csv('EMva/Output/sta/S_Ad1_rsd_mean.csv', header=True, index=True)

    # partial DEG
    if func == 'ggg_sd':
        A_D = pd.read_csv('EMva/Input/A_D_sd.csv', header=0, index_col=0)
        A_Dc = A_D.loc[S_Ac.columns, A_D.columns & S_Dc.columns]
        S_Dc = S_Dc.loc[:, A_Dc.columns]
        
        S_Ad0_sd, S_Ad1_se, prm = EM(
            S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
        S_Ad1_sd.to_csv('EMva/Output/S_Ad1_sd.csv', header=True, index=True)
        
    # all DEG
    if func == 'ggg_ad':
        A_D = pd.read_csv('EMva/Input/A_D_ad.csv', header=0, index_col=0)
        A_Dc = A_D.loc[S_Ac.columns, A_D.columns & S_Dc.columns]
        S_Dc = S_Dc.loc[:, A_Dc.columns]
        
        S_Ad0_ad, S_Ad1_sr, prm = EM(
            S_Ac, S_Dc, A_Dc, fix_mutation, e, cut_value, ite, cover_thres)
        S_Ad1_ad.to_csv('EMva/Output/S_Ad1_ad.csv', header=True, index=True)

In [ ]:
###################################################################################

In [135]:
# read data
gene_l = ['AKT1',
          'AR','ATM',
          'BRAF', 'RAF1', 'PDGFRA','KIT','KDR','CDK4',
          'ERBB2', 'EGFR','FGFR1','FGFR3',
          'MDM4', 'TP53',
          'TOP2A',
          'PPM1D',
          'PIK3CA',
          'MTOR']
S_Ac = pd.read_csv('EMva/Input/S_Ac_rd/S_Ac_rd_1.csv', header=0, index_col=0)[gene_l]
S_Dc = pd.read_csv('DrugNormal/Output/nGDSCexp.csv', header=0, index_col=0)

# S_Ac = pd.read_csv('EMva/Input/S_Am.csv', header=0, index_col=0)
# S_Dc = pd.read_csv('EMva/Input/S_Dm.csv', header=0, index_col=0)
# S_Dc.index = [int(ele) for ele in S_Dc.index]
A_D = pd.read_csv('EMva/Input/A_D.csv', header=0, index_col=0)

A_Dc = A_D.loc[A_D.index & S_Ac.columns, A_D.columns & S_Dc.columns]
S_Dc = S_Dc.loc[np.unique(S_Ac.index&S_Dc.index), A_Dc.columns]
S_Ac = S_Ac.loc[S_Dc.index, A_Dc.index]
S_Ac[S_Ac.isna()] = 0

S_Ac = S_Ac.reset_index().drop_duplicates(subset='index',keep='first').set_index('index')
S_Dc = S_Dc.reset_index().drop_duplicates(subset='index',keep='first').set_index('index')


In [132]:
dr_sen(S_Ac, S_Dc, A_Dc, func='ggg', fix_mutation=1)

In [ ]:
dr_sen(S_Ac, S_Dc, A_Dc, func='ggg_rad', fix_mutation=1)

In [ ]:
dr_sen(S_Ac, S_Dc, A_Dc, func='mut')

In [ ]:
pd.read_csv('EMva/Output/sta/S_Ad1_rad_mean.csv', header=0, index_col=0)

In [ ]:
pd.read_csv('EMva/Output/sta/S_Ac.csv', header=0, index_col=0)

In [133]:
pd.read_csv('EMva/Output/sta/S_Aggg1.csv', header=0, index_col=0)

,TP,Precision,Recall,F1
AKT1-0,44.0,0.156028,0.721311,0.256560
AKT1-1,34.0,0.109325,0.576271,0.183784
AKT1-2,86.0,0.275641,0.688000,0.393593
AKT1-3,87.0,0.317518,0.669231,0.430693
AKT1-4,35.0,0.514706,0.593220,0.551181
AR-5,4.0,0.018692,0.400000,0.035714
ATM-6,2.0,0.052632,0.076923,0.062500
ATM-7,15.0,0.348837,0.094937,0.149254
BRAF-8,3.0,0.048387,0.333333,0.084507
BRAF+RAF1-9,17.0,0.072650,0.629630,0.130268


In [ ]:
pd.read_csv('EMva/Output/sta/S_Aggg1.csv', header=0, index_col=0)